# Convert model to compute graph format

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [2]:
class MultiTaskModel(nn.Module):
    def __init__(
        self,
        # RNN input size, hidden size, and number of layers
        rnn_model,
        input_size,
        rnn_output_size,
        num_layers,
        # MLP hidden layers for each task
        state_pred_hidden_layers=[],
        installment_pred_hidden_layers=[],
        loan_pred_hidden_layers=[],
        recovery_rate_pred_hidden_layers=[],
        # Length of targets for each task
        len_state_targets=7,
        len_installment_targets=1,
        len_loan_targets=1,
        len_recovery_rate_targets=1,
    ):
        super(MultiTaskModel, self).__init__()
        # ------------------------------------
        # rnn Layer as a feature extractor
        if rnn_model == "RNN":
            self.rnn = nn.RNN(input_size, rnn_output_size, num_layers, batch_first=True)
        elif rnn_model == "LSTM":
            self.rnn = nn.LSTM(
                input_size, rnn_output_size, num_layers, batch_first=True
            )
        elif rnn_model == "GRU":
            self.rnn = nn.GRU(input_size, rnn_output_size, num_layers, batch_first=True)
        # ------------------------------------
        # MLP for prediction (state, installment, loan, recovery rate)
        # calculate the number of layers for each MLP
        state_pred_layers = (
            [rnn_output_size] + state_pred_hidden_layers + [len_state_targets]
        )
        installment_pred_layers = (
            [rnn_output_size]
            + installment_pred_hidden_layers
            + [len_installment_targets]
        )
        loan_pred_layers = (
            [rnn_output_size] + loan_pred_hidden_layers + [len_loan_targets]
        )
        recovery_rate_pred_layers = (
            [rnn_output_size]
            + recovery_rate_pred_hidden_layers
            + [len_recovery_rate_targets]
        )
        # create MLP layers
        state_mlp_layers = []
        installment_mlp_layers = []
        loan_mlp_layers = []
        recovery_rate_mlp_layers = []
        for i in range(len(state_pred_layers) - 1):
            state_mlp_layers.append(
                nn.Linear(state_pred_layers[i], state_pred_layers[i + 1])
            )
            if i < len(state_pred_layers) - 2:
                state_mlp_layers.append(nn.ReLU())
        for i in range(len(installment_pred_layers) - 1):
            installment_mlp_layers.append(
                nn.Linear(installment_pred_layers[i], installment_pred_layers[i + 1])
            )
            if i < len(installment_pred_layers) - 2:
                installment_mlp_layers.append(nn.ReLU())
            else:
                installment_mlp_layers.append(nn.Sigmoid())
        for i in range(len(loan_pred_layers) - 1):
            loan_mlp_layers.append(
                nn.Linear(loan_pred_layers[i], loan_pred_layers[i + 1])
            )
            if i < len(loan_pred_layers) - 2:
                loan_mlp_layers.append(nn.ReLU())
            else:
                loan_mlp_layers.append(nn.Sigmoid())
        for i in range(len(recovery_rate_pred_layers) - 1):
            recovery_rate_mlp_layers.append(
                nn.Linear(
                    recovery_rate_pred_layers[i], recovery_rate_pred_layers[i + 1]
                )
            )
            recovery_rate_mlp_layers.append(nn.ReLU())
        self.state_pred = nn.Sequential(*state_mlp_layers)
        self.installment_pred = nn.Sequential(*installment_mlp_layers)
        self.loan_pred = nn.Sequential(*loan_mlp_layers)
        self.recovery_rate_pred = nn.Sequential(*recovery_rate_mlp_layers)

    def forward(self, x, lengths):  # Accepts raw input and lengths
        # Pack sequence
        packed_x = pack_padded_sequence(
            x, lengths, batch_first=True, enforce_sorted=False
        )
        packed_out, _ = self.rnn(packed_x)
        rnn_out, _ = pad_packed_sequence(packed_out, batch_first=True)
        # Pass through MLP
        state_out = self.state_pred(rnn_out)
        installment_out = self.installment_pred(rnn_out)
        loan_out = self.loan_pred(rnn_out)
        recovery_rate_out = self.recovery_rate_pred(rnn_out)

        return state_out, installment_out, loan_out, recovery_rate_out



In [3]:
model = MultiTaskModel(
    rnn_model="GRU",
    input_size=21,
    rnn_output_size=64,
    num_layers=3,
    state_pred_hidden_layers=[32, 16],
    installment_pred_hidden_layers=[32, 16],
    loan_pred_hidden_layers=[32, 16],
    recovery_rate_pred_hidden_layers=[32, 16],
)

In [ ]:
model_path = "./Out/Model_81/Model_81.pth"

state_dict = torch.load(model_path, map_location=torch.device("cpu"))
model.load_state_dict(state_dict)
model.eval()

MultiTaskModel(
  (rnn): GRU(21, 64, num_layers=3, batch_first=True)
  (state_pred): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=7, bias=True)
  )
  (installment_pred): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=1, bias=True)
    (5): Sigmoid()
  )
  (loan_pred): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=1, bias=True)
    (5): Sigmoid()
  )
  (recovery_rate_pred): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16

In [5]:

scripted_model = torch.jit.script(model)
scripted_model.save("model_scripted.pt") 